In [3]:
import pickle
import time
import json
import cv2

In [4]:

index = 0
points_dict = {}
# actually 960 + 120 = 1080
for index in range(6572):
    with open(f"points-custom/{index}.pkl", "rb") as file:
        points = pickle.load(file)
        if index == 0:
            blank_points = points
        if len(points) == 0: # deal with the blank frames which I didn't add blank frames to
            points = blank_points
    # subtract offset from points
    points = [(int(point[0] - 60), int(point[1]-60)) for point in points]
    points_dict[index] = points

# output to text file using json
with open("bad-apple-unsorted.json", "w") as file:
    json.dump(points_dict, file)

In [19]:
# function to make video using the points from a json path

def make_video_from_json(json_path, fps, output_path):
    with open(json_path, "r") as file:
        points_dict = json.load(file)
    video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"XVID"), fps, (960, 720))
    for index in range(6572):
        print("Frame", index)
        frame = cv2.imread(f"frames/{index}.png")
        for point in points_dict[str(index)]:
            frame_c = frame.copy()
            # add green circle at point
            cv2.circle(frame_c, point, 10, (0, 255, 0), -1)
            video.write(frame_c)
    video.release()

In [16]:
make_video_from_json("bad-apple-unsorted.json", 120, "bad-apple-unsorted.avi")

In [ ]:
solves = {}


Now, I want to order the points for each frame using the travelling salesman problem heuristics.

In [13]:
# import tsp solver
# from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing
import numpy as np

# hash solves for each frame
for index, points in points_dict.items():
    pickle_save_path = f"points-tsp/{index}.pkl"
    if len(points) <= 2:
        with open(pickle_save_path, "wb") as file:
            pickle.dump(points, file)
        continue
    print(len(points))
    print("Solving for", index)
    # get a hash from the points
    points_hash = hash(tuple(points))
    if points_hash in solves:
        print(f"Skipping {index} because already solved")
        with open(pickle_save_path, "wb") as file:
            pickle.dump(solves[points_hash], file)
        continue
    # create distance matrix
    distance_matrix = []
    for i in range(len(points)):
        row = []
        for j in range(len(points)):
            # use euclidean distance
            row.append(((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)**0.5)
        distance_matrix.append(row)
    # solve tsp
    permutation, distance = solve_tsp_simulated_annealing(np.array(distance_matrix))
    # reorder points in points using permutation
    new_path = [points[i] for i in permutation]
    solves[points_hash] = new_path
    # save to pickle in points-tsp/index.pkl
    with open(pickle_save_path, "wb") as file:
        pickle.dump(new_path, file)


81
Solving for 0
Skipping 0 because already solved
81
Solving for 1
Skipping 1 because already solved
81
Solving for 2
Skipping 2 because already solved
81
Solving for 3
Skipping 3 because already solved
81
Solving for 4
Skipping 4 because already solved
81
Solving for 5
Skipping 5 because already solved
81
Solving for 6
Skipping 6 because already solved
81
Solving for 7
Skipping 7 because already solved
81
Solving for 8
Skipping 8 because already solved
81
Solving for 9
Skipping 9 because already solved
81
Solving for 10
Skipping 10 because already solved
81
Solving for 11
Skipping 11 because already solved
81
Solving for 12
Skipping 12 because already solved
81
Solving for 13
Skipping 13 because already solved
81
Solving for 14
Skipping 14 because already solved
81
Solving for 15
Skipping 15 because already solved
81
Solving for 16
Skipping 16 because already solved
81
Solving for 17
Skipping 17 because already solved
81
Solving for 18
Skipping 18 because already solved
81
Solving fo

In [23]:
# function to check if list of points is clockwise or anticlockwise
def is_counter_clockwise(points):
    summed = 0
    for i in range(len(points)):
        point1 = points[i]
        point2 = points[(i + 1) % len(points)]
        summed += (point2[0] - point1[0]) * (point2[1] + point1[1])  #(x2 − x1)(y2 + y1)
    return summed < 0

In [24]:
# gather from those pickles into a json file

index = 0
points_dict_tsp = {}
# actually 960 + 120 = 1080
for index in range(6572):
    with open(f"points-tsp/{index}.pkl", "rb") as file:
        points = pickle.load(file)
    if is_counter_clockwise(points):
        points = points[::-1]
    points_dict_tsp[index] = points


# output to text file using json
with open("bad-apple-tsp.json", "w") as file:
    json.dump(points_dict_tsp, file)

In [25]:
# make a video
make_video_from_json("bad-apple-tsp.json", 120, "bad-apple-json-tsp-clockwise.avi")

Frame 0
Frame 1
Frame 2
Frame 3
Frame 4
Frame 5
Frame 6
Frame 7
Frame 8
Frame 9
Frame 10
Frame 11
Frame 12
Frame 13
Frame 14
Frame 15
Frame 16
Frame 17
Frame 18
Frame 19
Frame 20
Frame 21
Frame 22
Frame 23
Frame 24
Frame 25
Frame 26
Frame 27
Frame 28
Frame 29
Frame 30
Frame 31
Frame 32
Frame 33
Frame 34
Frame 35
Frame 36
Frame 37
Frame 38
Frame 39
Frame 40
Frame 41
Frame 42
Frame 43
Frame 44
Frame 45
Frame 46
Frame 47
Frame 48
Frame 49
Frame 50
Frame 51
Frame 52
Frame 53
Frame 54
Frame 55
Frame 56
Frame 57
Frame 58
Frame 59
Frame 60
Frame 61
Frame 62
Frame 63
Frame 64
Frame 65
Frame 66
Frame 67
Frame 68
Frame 69
Frame 70
Frame 71
Frame 72
Frame 73
Frame 74
Frame 75
Frame 76
Frame 77
Frame 78
Frame 79
Frame 80
Frame 81
Frame 82
Frame 83
Frame 84
Frame 85
Frame 86
Frame 87
Frame 88
Frame 89
Frame 90
Frame 91
Frame 92
Frame 93
Frame 94
Frame 95
Frame 96
Frame 97
Frame 98
Frame 99
Frame 100
Frame 101
Frame 102
Frame 103
Frame 104
Frame 105
Frame 106
Frame 107
Frame 108
Frame 109
Frame 110
